# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#import csv file from output path
file_path = os.path.join("..", 'output_data', 'cities.csv')
cities_df = pd.read_csv(file_path)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps, add locations from cities dataframe, then add a heatmap.
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "Lng"]].astype(float)
fig = gmaps.figure(map_type="TERRAIN",center=(40.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=cities_df['Humidity'], 
                                 dissipating=False, max_intensity=cities_df['Humidity'].max().astype(float),
                                 point_radius=2)
fig.add_layer(heat_layer)
#Print a heatmap.
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#Create a new dataframe by filtering by preferred weather conditions
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
#Drop null values
hotel_df = cities_df.loc[cities_df["Max Temp"] > 75]
hotel_df = hotel_df.loc[cities_df["Max Temp"] < 80]
hotel_df = hotel_df.loc[cities_df["Wind Speed"] < 10]
hotel_df = hotel_df.loc[cities_df["Cloudiness"] == 0]
hotel_df = hotel_df.dropna(how='any')
print(f'Using these specific criteria has reduced the dataframe to only {len(hotel_df)} hotels.')
hotel_df

Using these specific criteria has reduced the dataframe to only 7 hotels.


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
57,57,san patricio,0,MX,1592280066,73,19.22,-104.70,78.530,2.25
125,125,poum,0,NC,1592279870,72,-20.23,164.02,75.362,6.30
196,196,aswan,0,EG,1592280232,27,24.09,32.91,78.638,4.55
221,221,panaba,0,MX,1592280207,91,21.28,-88.27,75.830,2.49
294,294,carnarvon,0,AU,1592280001,14,-24.87,113.63,77.000,6.20
340,340,sakakah,0,SA,1592280389,27,29.97,40.21,75.200,3.60
557,557,tamiahua,0,MX,1592280422,83,21.27,-97.45,76.298,0.51


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#Expand dataframe, init lists, base url
hotel_df['Hotel Name'] = hotel_df['Date']
cities_list = hotel_df["City"]
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
x=0
#loop through each city and find a hotel within 5000m of the co-ordinates. If no hotel, then exception.
for city in cities_list:
    #geocoordinates
    try:
        target_coordinates = (f'{hotel_df.iloc[x,6]}, {hotel_df.iloc[x,7]}')
        target_search = "Hotel"
        target_radius = 5000
        target_type = "lodging"  
        # set up a parameters dictionary
        params = {"location": target_coordinates,
            "keyword": target_search,
            "radius": target_radius,
            "type": target_type,
            "key": g_key}
        # run a request using params dictionary
        response = requests.get(base_url, params=params)
        places_data = response.json()
        hotel_df.iloc[x,10] = places_data["results"][0]["name"]
        #print(places_data["results"][0]["name"])
    except IndexError:
        #print(f'No hotel within {target_radius}m.')
        hotel_df.iloc[x,10] = (f'No hotel within {target_radius}m.')
    x = x + 1
#Print the new dataframe,including a column for nearest Hotel name.
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
57,57,san patricio,0,MX,1592280066,73,19.22,-104.70,78.530,2.25,Grand Isla Navidad Resort
125,125,poum,0,NC,1592279870,72,-20.23,164.02,75.362,6.30,No hotel within 5000m.
196,196,aswan,0,EG,1592280232,27,24.09,32.91,78.638,4.55,Sofitel Legend Old Cataract Aswan
221,221,panaba,0,MX,1592280207,91,21.28,-88.27,75.830,2.49,No hotel within 5000m.
294,294,carnarvon,0,AU,1592280001,14,-24.87,113.63,77.000,6.20,Hospitality Carnarvon
340,340,sakakah,0,SA,1592280389,27,29.97,40.21,75.200,3.60,Raoum Inn Hotel
557,557,tamiahua,0,MX,1592280422,83,21.27,-97.45,76.298,0.51,Mission Of The Sun Hotel


In [6]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [7]:
# Customize the size of the figure
figure_layout = {
    'width': '960px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,center=(50.0, +15), zoom_level=2)
# Add the layer to the map
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(layout=figure_layout,center=(40.0, -5.0), zoom_level=2)
fig.add_layer(marker_layer)
#Print a new map with just the hotels and info boxes.
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [8]:
#Same, but on top of the heat map.On a Satellite map, because why not?
fig = gmaps.figure(map_type="SATELLITE",center=(40.0, -5.0), zoom_level=2)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))